In [8]:
import os
import matplotlib
import vame
import numpy as np
import glob
import scipy
from vame.util import auxiliary as aux
from vame.analysis import behavior_structure as bs
from vame.custom import helperFunctions as hf
from vame.custom import ACWS_videowriter as avw
import polars as pl
import pandas as pd
from vame.util import auxiliary as aux
from vame.analysis import behavior_structure as bs
from vame.custom import helperFunctions as hf
from vame.custom import ALR_helperFunctions as AlHf
from vame.custom import ALR_statsFunctions as AlSt
from vame.custom import ALR_plottingFunctions as AlPf
from vame.custom import ACWS_videowriter as avw
from vame.custom import ALR_kinematics as kin
from vame.custom import ALR_analysis as an
from vame.analysis import community_analysis as ca
from importlib import reload
from icecream import ic
from scipy.stats import shapiro
import matplotlib.pyplot as plt
import six


The loaded data frame (df) was made using the file: "/Volumes/G-DRIVE_SSD/Open_Arena_Results/make_motif_usage_summed.py"

df_master was made using the file: "/Users/adanredwine/VAME/vame/custom/ALR_work_files/ALR_Behavior_Data_Approach_Final.py"
The data was made using the HMM recognizing 40 motifs over 600 iterations.

In [96]:
df_master = pd.read_csv("/Volumes/G-DRIVE_SSD/Open_Arena_Results/ALR_master_df.csv")

# Only keep the columns needed for information: 'Motif', 'Cluster', 'Predominant_Behavior
df_info = df_master[['Motif', 'Cluster', 'Predominant_Behavior']].drop_duplicates()

df = pd.read_csv("/Volumes/G-DRIVE_SSD/Open_Arena_Results/ALR_motif_usage_summed.csv")

# View the columns in the data 
df.columns

Index(['Animal_ID', 'Group', 'Time_Point', 'Motif', 'Frequency'], dtype='object')

In [98]:
df_info

,Motif,Cluster,Predominant_Behavior
0,23,3.0,Sniffing
23,1,3.0,Sniffing
44,28,2.0,Rearing
51,38,5.0,Walking
52,30,NaN,NaN
63,24,3.0,Sniffing
69,12,2.0,Rearing
84,2,3.0,Sniffing
96,19,NaN,NaN
170,0,2.0,Rearing


# Checking Normality of Data
## Non-transformed Data

In [30]:
normality_results = []

for group in df["Group"].unique():
    for time_point in df["Time_Point"].unique():
        for motif in df["Motif"].unique():
            data = df[(df["Group"] == group) & (df["Time_Point"] == time_point) & (df["Motif"] == motif)]["Frequency"]
            
            if len(data) > 3:
                stat, p = shapiro(data)
                normality_results.append(
                        {
                            "Motif": motif,
                            "Group": group,
                            "Time_Point": time_point,
                            "W": stat,
                            "p": p,
                        }
                    )

# Convert the results to a dataframe
normality_results_df = pd.DataFrame(normality_results)

normality_results_df.head()

,Motif,Group,Time_Point,W,p
0,12,Sham,0,0.950904,0.655523
1,22,Sham,0,0.935653,0.470822
2,20,Sham,0,0.975376,0.935054
3,29,Sham,0,0.952446,0.675324
4,9,Sham,0,0.553487,0.000007


In [31]:
# Check the percentage of significant results
normality_results_df["p"].apply(lambda x: x < 0.05).value_counts(normalize=True)


p
False    0.695971
True     0.304029
Name: proportion, dtype: float64

This output means that about 30% of the data is not normally distributed. This is not ideal, but also not unusable. Next we will see if a log transformation will help.

## Log Transformed Data

In [114]:
# log transform the data
df["Log_Frequency"] = np.log(df["Frequency"])

normality_results_log = []

for group in df["Group"].unique():
    for time_point in df["Time_Point"].unique():
        for motif in df["Motif"].unique():
            data = df[(df["Group"] == group) & (df["Time_Point"] == time_point) & (df["Motif"] == motif)]["Log_Frequency"]
            
            if len(data) > 3:
                stat, p = shapiro(data)
                normality_results_log.append(
                        {
                            "Motif": motif,
                            "Group": group,
                            "Time_Point": time_point,
                            "W": stat,
                            "p": p,
                        }
                    )
                
# Convert the results to a dataframe
normality_results_log_df = pd.DataFrame(normality_results_log)

normality_results_log_df.head()            


,Motif,Group,Time_Point,W,p
0,12,Sham,0,0.935160,0.465368
1,22,Sham,0,0.942675,0.552443
2,20,Sham,0,0.937320,0.489521
3,29,Sham,0,0.802935,0.010282
4,9,Sham,0,0.819878,0.017183


In [37]:
# Check the percentage of significant results
normality_results_log_df["p"].apply(lambda x: x < 0.05).value_counts(normalize=True)


p
False    0.901832
True     0.098168
Name: proportion, dtype: float64

The 90% of the data is normally distributed when using a log transformation. The analysis will be done using the log transformed data.

# Question 1

In [121]:
# Filter the data to only include Injured and Sham groups at Time_Points less than 11
df_filtered = df[(df["Group"].isin(["Injured", "Sham"])) & (df["Time_Point"] < 11)]

#normality_res_filtered = normality_results_df[(normality_results_df["Group"].isin(["Injured", "Sham"])) & (normality_results_df["Time_Point"] < 11)]
normality_res_filtered = normality_results_log_df[(normality_results_log_df["Group"].isin(["Injured", "Sham"])) & (normality_results_log_df["Time_Point"] < 11)]

In [122]:
# Check the percentage of significant results
normality_res_filtered["p"].apply(lambda x: x < 0.05).value_counts(normalize=True)


p
False    0.923077
True     0.076923
Name: proportion, dtype: float64

In [123]:
normality_res_filtered

,Motif,Group,Time_Point,W,p
0,12,Sham,0,0.935160,0.465368
1,22,Sham,0,0.942675,0.552443
2,20,Sham,0,0.937320,0.489521
3,29,Sham,0,0.802935,0.010282
4,9,Sham,0,0.819878,0.017183
...,...,...,...,...,...
580,38,Injured,8,0.914875,0.246242
581,1,Injured,8,0.951616,0.660719
582,18,Injured,8,0.800420,0.009473
583,14,Injured,8,0.955525,0.714910


## Difference in Motif Usage Between Groups at Each Time Point

In [124]:
# Create a dictionary to store the results
import scipy
GroupWiseResults = []

for motif in df_filtered["Motif"].unique():
    for tp in df_filtered["Time_Point"].unique():
        
        normality_res_filtered_tp = normality_res_filtered[normality_res_filtered["Time_Point"] == tp]
        normality_res_filtered_tp = normality_res_filtered_tp[normality_res_filtered_tp["Motif"] == motif]
        
        if len(normality_res_filtered_tp) < 2:
            print("Not enough data for motif: ", motif, " at time point: ", tp)
            continue
        
        if (normality_res_filtered_tp["p"].iloc[0] <= 0.05 and normality_res_filtered_tp["p"].iloc[1] <= 0.05):
            data = df_filtered[(df_filtered["Time_Point"] == tp) & (df_filtered["Motif"] == motif)]
            
            # Perform a Mann-Whitney U test
            stat, p = scipy.stats.mannwhitneyu(data[data["Group"] == "Injured"]["Log_Frequency"], data[data["Group"] == "Sham"]["Log_Frequency"])
            
            GroupWiseResults.append({
                "Motif": motif,
                "Time_Point": tp,
                "Stat": stat,
                "p": p,
                "test": "Mann-Whitney U"
            })
        
        elif ((normality_res_filtered_tp["p"].iloc[0] > 0.05 and normality_res_filtered_tp["p"].iloc[1]) > 0.05):
            data = df_filtered[(df_filtered["Time_Point"] == tp) & (df_filtered["Motif"] == motif)]
            
            stat, p = scipy.stats.ttest_ind(data[data["Group"] == "Injured"]["Frequency"], data[data["Group"] == "Sham"]["Frequency"])
            
            # Perform a t-test
            stat, p = scipy.stats.ttest_ind(data[data["Group"] == "Injured"]["Log_Frequency"], data[data["Group"] == "Sham"]["Log_Frequency"])
            
            GroupWiseResults.append({
                "Motif": motif,
                "Time_Point": tp,
                "Stat": stat,
                "p": p,
                "test": "t-test"
            })

            
        

Not enough data for motif:  31  at time point:  0
Not enough data for motif:  31  at time point:  2
Not enough data for motif:  31  at time point:  4
Not enough data for motif:  31  at time point:  6
Not enough data for motif:  31  at time point:  8


**NOTE:** There is not enough data for motif 31 to be included in the analysis. 

In [125]:
from statsmodels.stats.multitest import multipletests
# Convert the results to a dataframe
GroupWiseResults_df = pd.DataFrame(GroupWiseResults)

# Perform a multiple testing correction
GroupWiseResults_df["p_corrected"] = multipletests(GroupWiseResults_df["p"], method="bonferroni")[1]

GroupWiseResults_df.head()

,Motif,Time_Point,Stat,p,test,p_corrected
0,12,0,1.225435,0.233978,t-test,1.0
1,12,2,0.378001,0.709224,t-test,1.0
2,12,4,0.050490,0.960209,t-test,1.0
3,12,6,1.534063,0.139943,t-test,1.0
4,12,8,0.457930,0.651706,t-test,1.0


In [126]:
# Add the corrected p-values to the dataframe df_filtered
df_filtered = df_filtered.merge(GroupWiseResults_df, on=["Motif", "Time_Point"], how="left")

df_filtered.head()
#AlPf.plot_motifs_all_time_points(df_long_logNorm_sham, comparison = 'Group')

,Animal_ID,Group,Time_Point,Motif,Frequency,Log_Frequency,Stat,p,test,p_corrected
0,I2,Sham,0,12,1019,6.926577,1.225435,0.233978,t-test,1.0
1,E1,Injured,0,12,1002,6.909753,1.225435,0.233978,t-test,1.0
2,T1,Sham,0,12,953,6.859615,1.225435,0.233978,t-test,1.0
3,O2,Injured,0,12,1231,7.115582,1.225435,0.233978,t-test,1.0
4,L1,Sham,0,12,1022,6.929517,1.225435,0.233978,t-test,1.0


In [127]:
df_filtered.columns


Index(['Animal_ID', 'Group', 'Time_Point', 'Motif', 'Frequency',
       'Log_Frequency', 'Stat', 'p', 'test', 'p_corrected'],
      dtype='object')

In [139]:
import importlib
importlib.reload(AlPf)
# Merge df_filtered with df_info
df_filtered_plot = df_filtered.merge(df_info, on=["Motif"], how="left")

# Rename 'p_corrected' to 'P_Value_Group' for plotting
df_filtered_plot = df_filtered_plot.rename(columns={"p_corrected": "P_Value_Group"})

# Normalize the data to the Sham group at each time point to make the data more interpretable visually
# Assuming that 'Time_Point' is the column that represents the time point
grouped = df_filtered_plot.groupby('Time_Point')

def normalize_to_sham(group):
    sham_frequency = group[group['Group'] == 'Sham']['Log_Frequency'].mean()
    group['Log_Frequency_Normalized'] = group['Log_Frequency'] - sham_frequency
    return group

df_filtered_plot = grouped.apply(normalize_to_sham)

# Set 'Log_Frequency_Normalized' to 0 for Sham group
df_filtered_plot.loc[df_filtered_plot['Group'] == 'Sham', 'Log_Frequency_Normalized'] = 0

AlPf.plot_motifs_all_time_points2(df_filtered_plot, comparison = 'Group', value_col = 'Log_Frequency_Normalized')

/var/folders/65/09b4rkk97lq0vlwt6bbgpkkh0000gn/T/ipykernel_40196/1131407091.py:18: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_filtered_plot = grouped.apply(normalize_to_sham)


KeyError: 2.0

In [143]:
import seaborn as sns

# Set the backend so that the plots are displayed inline
for t in df_filtered_plot["Time_Point"].unique():
    df_filtered_plot_t = df_filtered_plot[df_filtered_plot["Time_Point"] == t]
    
    # Make line plot with x = Motif, y = Log_Frequency_Normalized, hue = Group
    plt.figure(figsize=(20, 10))
    sns.lineplot(data=df_filtered_plot_t, x="Motif", y="Log_Frequency_Normalized", hue="Group")
    plt.title(f"Time Point: {t}")
    plt.xticks(rotation=90)
    plt.show()

ImportError: Cannot load backend 'TkAgg' which requires the 'tk' interactive framework, as 'qt' is currently running